## 1절. 연관분석 개요
### 1.1. 연관분석
### 1.2. 연관분석 평가

- pip install apyori 
- 데이터 수가 많으면 메모리를 많이 잡아먹는 경우가 있음. 
- 지지도 : 전체 트랜젝션에서 항목 집합 (x,y)를 포함하는 트랜잭션의 비율 
- 신뢰도 : x 를 포함하는 트랜잭션 중 y도 포함하는 트랜잭션의 비율 
- 향상도 : lift = p(y|x)/p(y) = 신뢰도 / p(y) - 관계파악 
- 파이계수
- IS 측도 

## 2절. 트랜잭션 데이터
- 트랜잭션 : 서로 관련있는 데이터의 모음임. 
- 연관분석시 트랜잭션으로 부터 연관규칙을 도출함. 

### 2.1. CSV파일로부터 트랜잭션 데이터 생성

In [2]:
import csv
with open("basket.csv",'r',encoding="UTF8") as cf:
    transactions = []
    r = csv.reader(cf) # pd로 읽으면 df이 되기때문에 csv로 불러와야함
    for row in r:
        transactions.append(row)

In [3]:
transactions

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['콜라', '맥주', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

## 연관분석
### 3.1. 연관규칙 생성
- apriori(transactions(리스트의 리스트), **kwargs)

In [4]:
from apyori import apriori 
rules = apriori(transactions,min_support = 0.1, min_confidence=0.1)
print(rules)
results = list(rules)
type(results)

<generator object apriori at 0x00000000051412C8>


list

In [6]:
results[0]

# frozenset 

RelationRecord(items=frozenset({'맥주'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'맥주'}), confidence=0.4, lift=1.0)])

In [7]:
results[10]

RelationRecord(items=frozenset({'소주', '콜라'}), support=0.6, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'소주', '콜라'}), confidence=0.6, lift=1.0), OrderedStatistic(items_base=frozenset({'소주'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'소주'}), confidence=0.7499999999999999, lift=1.2499999999999998)])

### 3.2. 연관 규칙 조회

In [19]:
print("lhs => rhs \ tsupport \ tconfidence \ tlift")
for row in results :
    support = row[1]
    ordered_stat = row[2]
    for ordered_item in ordered_stat:
        lhs = [x for x in ordered_item[0]]
        rhs = [x for x in ordered_item[1]]
        confidence = ordered_item[2]
        lift = ordered_item[3]
        print(lhs, "=>", rhs, "\t{:>5.4f}\t{:>5.4f}\t{:>5.4f}".\
             format(support,confidence,lift))

lhs => rhs \ tsupport \ tconfidence \ tlift
[] => ['맥주'] 	0.4000	0.4000	1.0000
[] => ['소주'] 	0.6000	0.6000	1.0000
[] => ['오렌지주스'] 	0.4000	0.4000	1.0000
[] => ['와인'] 	0.6000	0.6000	1.0000
[] => ['콜라'] 	0.8000	0.8000	1.0000
[] => ['맥주', '소주'] 	0.2000	0.2000	1.0000
['맥주'] => ['소주'] 	0.2000	0.5000	0.8333
['소주'] => ['맥주'] 	0.2000	0.3333	0.8333
[] => ['맥주', '와인'] 	0.2000	0.2000	1.0000
['맥주'] => ['와인'] 	0.2000	0.5000	0.8333
['와인'] => ['맥주'] 	0.2000	0.3333	0.8333
[] => ['맥주', '콜라'] 	0.4000	0.4000	1.0000
['맥주'] => ['콜라'] 	0.4000	1.0000	1.2500
['콜라'] => ['맥주'] 	0.4000	0.5000	1.2500
[] => ['소주', '오렌지주스'] 	0.2000	0.2000	1.0000
['소주'] => ['오렌지주스'] 	0.2000	0.3333	0.8333
['오렌지주스'] => ['소주'] 	0.2000	0.5000	0.8333
[] => ['와인', '소주'] 	0.2000	0.2000	1.0000
['소주'] => ['와인'] 	0.2000	0.3333	0.5556
['와인'] => ['소주'] 	0.2000	0.3333	0.5556
[] => ['소주', '콜라'] 	0.6000	0.6000	1.0000
['소주'] => ['콜라'] 	0.6000	1.0000	1.2500
['콜라'] => ['소주'] 	0.6000	0.7500	1.2500
[] => ['와인', '오렌지주스'] 	0.2000	0.2000	1.0000
['오렌지주스'] =

## 4절

### 4.3. 기사 수집 및 형태소 분석 

In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()

import requests 
from bs4 import BeautifulSoup
url = 'http://fs.jtbc.joins.com/RSS/economy.xml'
response = requests.get(url)
soup = BeautifulSoup(response.content,"xml")
link_list = soup.select('item link')

news = []
for link in link_list:
    news_url = link.text
    news_response = requests.get(news_url)    
    news_soup = BeautifulSoup(news_response.content,"html.parser")
    news_content = news_soup.select_one(".article_content")
    news_n = kkma.nouns(news_content.text)
    news_nouns = filter(lambda word : len(word)>1 , news_n)
    news.append(list(news_nouns))
    
result = []
for i in range(len(news)):
    result.extend(news[i])
result= " ".join(result)

In [5]:
result

'인용 인용보도 보도 프로그램 프로그램명 아침 저작권 방송 07 00 08 30 진행 황남희 남희 앵커 현재 상황 코로나 여파 일자리 문제 지금 심상 심상치가 치가 위기 국민 고용 고용보험 보험 필요 필요하다 하다 얘기 재인 대통령 전국민 안전망 계획 오늘 12 12일 전문가 정철 경제 경제평론가 평론가 시대 기초 급물살일용직 물살 일용직 프리랜서 고용보험국민취업 취업 지원 고용보험법 보험법 개정안 환노위 통과 통과예술인 예술인 실업 실업급여 급여 첫발 첫발보험설계사 설계사 특수 특수고용노동자 노동자 제외 제외보험료 보험료 부담 소상공인 상공인 자영업자 우려 우려전 대상 확대 해외 사례 공식화 과제 재원 마련 방안 자세한 세한 내용 영상 확인 정유 정유업계 업계 적자 4조 쇼크 현실화 현실화1분기 분기 영업 영업손실 손실 사상 최악 정유사 4곳 코로나 사태 국제 유가 폭락 영향 1위 업체 노베이션 베이 1조 7000 7000억 원대 칼텍스 텍스 에쓰 에쓰오일 오일 초반 초반대의 대의 충격 5월 수출 46 10 10일 수출금액 금액 지난해 기간 50 관세청 조사 결과 지난달 이후 정도 승용 승용차가 차가 80 석유 석유제품 제품 75 감소 수도권 광역시 분양권 전매 금지 금지서울 서울 경기 일부 지역 시행 금지가 지가 전역 지방 적용 8월 최근 인천 시흥 등에 투기 정부 규제 조치 확대 앵커 긴급 재난 재난지원금 지원금 신청 어제 11 11일 시작 신용 신용카드 카드 체크 충전 사람 우선 대상 하루 동안 171 171만 가구 1조 1500 1500억 가량 이재 이재승 기자 보도 긴급재난지원금 어디 시민 김세진 세진 인천 청라국제도시 국제 도시 학원비 번째 마트 그다음 미용실 임영대 영대 경기 고양 고양시 가정 생활 생활용품 용품 반찬 학원 음식 음식재료 재료 동네 슈퍼 주유 주유소 편의점 병원 약국 사용 대형 대형마트 원칙적 지역 지역경제 경제 골목 골목상권 상권 취지 안의 안경점 임대 임대매장 매장 프랜차이즈 이즈 라도 본사 카드사 9곳 홈페이지 애플리케이션 

### 4.4. 연관분석 

In [17]:
from apyori import apriori
rules = apriori(news, min_support = 0.5, min_confidence = 0.5)
results = list(rules)

KeyboardInterrupt: 

In [ ]:
import pandas as pd
result_df = pd.DataFrame(None, columns = ["lhs","rhs","support","confidence","lift"])
index = 0
for row in results:
    support  = row[1]
    ordered_stat = row[2]
    for ordered_item in ordered_stat:
        lhs = " ".join(x.strip() for x in ordered_item[0])
        rhs = " ".join(x.strip() for x in ordered_item[1])
        confidence = ordered_item[2]
        lift = ordered_item[3]
        result_df.loc[index] = [lhs,rhs,support,confidence,lift]
        index = index +1 

### 4.5. 연관분석 탐색

In [ ]:
result_df.loc[result_df.lhs==""].sort_values(by=["support"],  asending=False)


In [ ]:
result_df.loc[(result_df.lhs.str.contains("택시"))&
             (result_df.rhs == "공유")].sort_values(by=["lift"], ascending= False )

In [ ]:
result_df.loc[(result_df.lhs.str.contains("택시"))&
             (result_df.rhs == "공유")].sort_values(by=["support"], ascending= False)